Query USPTO PatentsView SQLite database

In [1]:
import sqlite3
import re
import json
import os
import random

import matplotlib.pyplot as plt
import numpy as np

from icecream import ic

In [2]:
#timer class
import time

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        self._start_time = time.perf_counter()

    def stop(self):
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
#         print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return elapsed_time

t = Timer()

In [3]:
db_path = os.path.expanduser("~/goa/patent_data.sqlite3")

In [4]:
db = sqlite3.connect(db_path)
c = db.cursor()

In [5]:
c.execute('PRAGMA TABLE_INFO({})'.format("patent"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)


Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'id', 'TEXT', 0, None, 0)
(1, 'type', 'TEXT', 0, None, 0)
(2, 'number', 'TEXT', 0, None, 0)
(3, 'country', 'TEXT', 0, None, 0)
(4, 'date', 'TEXT', 0, None, 0)
(5, 'abstract', 'TEXT', 0, None, 0)
(6, 'title', 'TEXT', 0, None, 0)
(7, 'kind', 'TEXT', 0, None, 0)
(8, 'num_claims', 'TEXT', 0, None, 0)
(9, 'filename', 'TEXT', 0, None, 0)
(10, 'withdrawn', 'TEXT', 0, None, 0)


In [6]:
# get a single entry from patent table
c.execute('''
    SELECT * 
    FROM patent 
    ORDER BY ROWID ASC 
    LIMIT 1
''')
rows = c.fetchall()
for row in rows:
    print("-" * 10)
    for r in row:
        print(r)

----------
10000000
utility
10000000
US
2018-06-19
A frequency modulated (coherent) laser detection and ranging system includes a read-out integrated circuit formed with a two-dimensional array of detector elements each including a photosensitive region receiving both return light reflected from a target and light from a local oscillator, and local processing circuitry sampling the output of the photosensitive region four times during each sample period clock cycle to obtain quadrature components. A data bus coupled to one or more outputs of each of the detector elements receives the quadrature components from each of the detector elements for each sample period and serializes the received quadrature components. A processor coupled to the data bus receives the serialized quadrature components and determines an amplitude and a phase for at least one interfering frequency corresponding to interference between the return light and the local oscillator light using the quadrature components

### Search patent title and abstract for keyword

In [133]:
keyword_sql = '''
    SELECT *
    FROM patent
    LEFT JOIN patent_inventor ON patent.id = patent_inventor.patent_id
    LEFT JOIN inventor ON patent_inventor.inventor_id = inventor.id
    LEFT JOIN patent_assignee ON patent.id = patent_assignee.patent_id
    LEFT JOIN assignee ON patent_assignee.assignee_id = assignee.id
    LEFT JOIN location ON patent_assignee.location_id = location.id
    WHERE title LIKE ? OR abstract LIKE ?
'''

In [134]:
# search terms, case insensitive
# followed by number of results
# search is not smart, i.e. needs to match exactly

keyword = "automation" # 21958
# keyword = "warehouse" # 4813
# keyword = "logistics" # 2131
# keyword = "automated storage" # 839
# keyword = "automated retrieval" # 98
# keyword = "automated guided vehicle" # 480
# keyword = "order picking" # 142
# keyword = "order picking system" # 36
# keyword = "order picking station" # 7
# keyword = "fulfillment system" # 494

In [135]:
t.start()

search_term = f'%{keyword}%'
c.execute(keyword_sql, (search_term, search_term))

rows = c.fetchall()
t.stop()
print(f'Keyword: {keyword}')
print(f'Total number of results: {len(rows)}')

Elapsed time: 20.8819 seconds
Keyword: automation
Total number of results: 21958


In [136]:
names = [description[0] for description in c.description]
for row in rows[:3]:
    print("*** PATENT RECORD ***")
    for i, r in enumerate(row):
#         print(f'{names[i]}: {r}')
        line_new = '{:<12} : {:<12}'.format(names[i], r)
        print(line_new)
    print("-" * 30)

*** PATENT RECORD ***
id           : 10000968    
type         : utility     
number       : 10000968    
country      : US          
date         : 2018-06-19  
abstract     : An electromechanical actuator (11) includes a sealing element (21) provided with a housing (22), an electronic control unit (15) electrically connected to a first electrical connector (23), and a power supply cable (18) including, at one of the ends of same, a second electrical connector (24). The second connector (24) includes an elastic tab (26) provided with a stop element (27) cooperating with a recess (28) provided in the housing (22), and a first bending area (32) provided at the junction between a first end (26a) of the elastic tab (26) and a body (25) of the second connector (24). The stop element (27) is provided at a second free end (26b) of the elastic tab (26). The elastic tab (26) includes a second bending area (33) provided at the junction between an arm (37) and the stop element (27) of the elasti

Get the number of citations to foreign patents from US patents, totals by country

In [123]:
for row in rows:
    print(row)

(None, 0)
('000', 1)
('061', 1)
('081', 1)
('0E', 4)
('0L', 1)
('0O', 1)
('0P', 3)
('0R', 1)
('0T', 1)
('0U', 2)
('100', 1)
('109', 1)
('115', 1)
('120', 1)
('125', 1)
('151', 1)
('210', 1)
('37', 1)
('3P', 2)
('44.', 1)
('47', 1)
('? 7', 1)
('AD', 13)
('AE', 49)
('AF', 12)
('AG', 16)
('AI', 10)
('AIX       ', 1)
('AL', 141)
('ALX       ', 1)
('AM', 98)
('AMX       ', 44)
('AN', 16)
('AO', 28)
('AP', 101)
('AQ', 3)
('AR', 1626)
('AS', 1)
('AT', 23111)
('ATX       ', 681)
('AU', 141549)
('AU        ', 17)
('AUX       ', 3668)
('AW', 4)
('AX', 2)
('AZ', 4)
('Al', 3)
('Aus', 1)
('BA', 19)
('BB', 6)
('BD', 27)
('BE', 24906)
('BE        ', 8)
('BEX       ', 863)
('BF', 4)
('BG', 359)
('BGX       ', 3)
('BH', 11)
('BI', 1)
('BJ', 4)
('BM', 3)
('BN', 3)
('BO', 6)
('BQ', 10)
('BR', 7648)
('BRX       ', 53)
('BS', 11)
('BT', 1)
('BU', 2)
('BV', 1)
('BW', 1)
('BX', 182)
('BXX       ', 53)
('BY', 151)
('BYX       ', 1)
('BZ', 11)
('BZX       ', 1)
('C', 1)
('CA', 353033)
('CA        ', 53)
('CAX 

In [11]:
c.execute('''
    SELECT patent.id
    FROM patent
    LEFT JOIN patent_assignee ON patent.id = patent_assignee.patent_id
    LEFT JOIN assignee ON patent_assignee.assignee_id = assignee.id
    LEFT JOIN patent_inventor ON patent.id = patent_inventor.patent_id
    LEFT JOIN inventor ON patent_inventor.inventor_id = inventor.id
''')
print(len(rows))

1


In [12]:
print(rows)

[('10000000', 'utility', '10000000', 'US', '2018-06-19', 'A frequency modulated (coherent) laser detection and ranging system includes a read-out integrated circuit formed with a two-dimensional array of detector elements each including a photosensitive region receiving both return light reflected from a target and light from a local oscillator, and local processing circuitry sampling the output of the photosensitive region four times during each sample period clock cycle to obtain quadrature components. A data bus coupled to one or more outputs of each of the detector elements receives the quadrature components from each of the detector elements for each sample period and serializes the received quadrature components. A processor coupled to the data bus receives the serialized quadrature components and determines an amplitude and a phase for at least one interfering frequency corresponding to interference between the return light and the local oscillator light using the quadrature com

In [ ]:
c.execute('''
    SELECT count(patent.id)
    FROM patent
    LEFT JOIN patent_assignee ON patent.id = patent_assignee.patent_id
    LEFT JOIN assignee ON patent_assignee.assignee_id = assignee.id
    INNER JOIN patent_inventor ON patent.id = patent_inventor.patent_id
    INNER JOIN inventor ON patent_inventor.inventor_id = inventor.id
''')
print(rows)
#     LEFT JOIN location ON patent_assignee.location_id = location.id


Search by assignee

Amazon: Kiva Systems (now Amazon Robotics but there are no 'Amazon Robotics' assignees, there is Amazon Technology), Canvas Technology (machine vision) - all acquired by Amazon

Alibaba: Cainiao, Quicktron, GeekPlus, ALOG (another company name is Xinyi Technology)

Not Amazon or Alibaba: 
Fetch Robotics (now Zebra Technology) used by DHL
Hai Robotics, China - used by DHL and Booktopia; also used in South Korea 
Locus Robotics, US - DHL
OTTO Motors
Fabric (micro fulfillment) 

// count patents
join assignee: 8056486
join inventor: 8056486

In [26]:
assignee_sql = '''
    SELECT *
    FROM patent
    LEFT JOIN patent_assignee ON patent.id = patent_assignee.patent_id
    LEFT JOIN assignee ON patent_assignee.assignee_id = assignee.id
    LEFT JOIN location ON patent_assignee.location_id = location.id
    WHERE assignee.organization LIKE ? 
'''   

#     LEFT JOIN patent_inventor ON patent.id = patent_inventor.patent_id
#     LEFT JOIN inventor ON patent_inventor.inventor_id = inventor.id

In [27]:
assignee = "kiva Systems" # 15
# assignee = "amazon robotics" # 0
# assignee = "amazon technologies" # 14863 - e62cea34-5174-4cfd-a59d-7ebea8b78dca
# assignee = "canvas technology" # 6

# assignee = "cainiao" # 9
# assignee = "quicktron" # 0
# assignee = "geekplus" # 7
# assignee = "ALOG"
# assignee = "xinyi technology" # 1

# assignee = "fetch robotics"
# assignee = "zebra technology" 
# assignee = "hai robotics" 
# assignee = "locus robotics"
# assignee = "otto motors"
# assignee = "fabric"

In [28]:
t.start()

search_term = f'%{assignee}%'
c.execute(assignee_sql, (search_term,))

rows = c.fetchall()
end_time = t.stop()
print(f"Elapsed time: {end_time:0.4f} seconds")
print(f'Assignee: {assignee}')
print(f'Total number of results: {len(rows)}')

Elapsed time: 35.5887 seconds
Assignee: Kiva Systems
Total number of results: 15


Search time approx 35 s

In [29]:
names = [description[0] for description in c.description]
records = rows
# random.shuffle(records)
for row in records[:10]:
    print("-" * 30)
    print("*** PATENT RECORD ***")
    print("-" * 30)
    for i, r in enumerate(row):
#         print(f'{names[i]}: {r}')
        line_new = '{:<12} : {:<12}'.format(names[i], r)
        print(line_new)

------------------------------
*** PATENT RECORD ***
------------------------------
id           : 7402018     
type         : utility     
number       : 7402018     
country      : US          
date         : 2008-07-22  
abstract     : A system for transporting inventory includes an inventory holder and a mobile drive unit. The inventory holder includes a frame capable of storing inventory items and a docking plate capable of receiving a docking head from underneath. The mobile drive unit, includes a docking head capable of coupling to the docking plate and a drive module capable of propelling the mobile drive unit. The mobile drive unit is further capable to move the inventory holder when the docking head is coupled to the inventory holder.
title        : Inventory system with mobile drive unit and inventory holder
kind         : B2          
num_claims   : 20          
filename     : ipg080722.xml
withdrawn    : 0           
patent_id    : 7402018     
assignee_id  : 3415fa43-5835

### virtual table search

For performing full text search across multiple fields. Code currently incomplete.

In [ ]:
# virtual table search
t.start()
sql = '''SELECT * FROM patent_text WHERE patent_text MATCH "automation"'''
c.execute(sql)
rows = c.fetchall()
t.stop()
ic(len(rows))

In [ ]:
for row in rows[:3]:
    for r in row:
        print(r)
    print("-" * 10)

In [28]:
for row in rows:
    print(row)

(None, 0)
('000', 1)
('061', 1)
('081', 1)
('0E', 4)
('0L', 1)
('0O', 1)
('0P', 3)
('0R', 1)
('0T', 1)
('0U', 2)
('100', 1)
('109', 1)
('115', 1)
('120', 1)
('125', 1)
('151', 1)
('210', 1)
('37', 1)
('3P', 2)
('44.', 1)
('47', 1)
('? 7', 1)
('AD', 13)
('AE', 49)
('AF', 12)
('AG', 16)
('AI', 10)
('AIX       ', 1)
('AL', 141)
('ALX       ', 1)
('AM', 98)
('AMX       ', 44)
('AN', 16)
('AO', 28)
('AP', 101)
('AQ', 3)
('AR', 1626)
('AS', 1)
('AT', 23111)
('ATX       ', 681)
('AU', 141549)
('AU        ', 17)
('AUX       ', 3668)
('AW', 4)
('AX', 2)
('AZ', 4)
('Al', 3)
('Aus', 1)
('BA', 19)
('BB', 6)
('BD', 27)
('BE', 24906)
('BE        ', 8)
('BEX       ', 863)
('BF', 4)
('BG', 359)
('BGX       ', 3)
('BH', 11)
('BI', 1)
('BJ', 4)
('BM', 3)
('BN', 3)
('BO', 6)
('BQ', 10)
('BR', 7648)
('BRX       ', 53)
('BS', 11)
('BT', 1)
('BU', 2)
('BV', 1)
('BW', 1)
('BX', 182)
('BXX       ', 53)
('BY', 151)
('BYX       ', 1)
('BZ', 11)
('BZX       ', 1)
('C', 1)
('CA', 353033)
('CA        ', 53)
('CAX 

In [30]:
t.start()
c.execute('''
    SELECT location.country, COUNT(location.country)
    FROM patent
    LEFT JOIN patent_inventor ON patent.id = patent_inventor.patent_id
    LEFT JOIN inventor ON patent_inventor.inventor_id = inventor.id
    LEFT JOIN location ON patent_inventor.location_id = location.id
    GROUP BY location.country
''')

rows = c.fetchall()
ic(len(rows))
t.stop()

ic| len(rows): 267


Elapsed time: 114.8408 seconds


In [ ]:
for row in rows:
    print(row)